In [3]:
import numpy as np
import pandas as pd
import copy, math

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
passenger_ids = test['PassengerId']

In [8]:
train = train.drop(columns=['Name', 'PassengerId'])
test = test.drop(columns=['Name', 'PassengerId'])

In [9]:
cols = [
    'Age',
    'Fare'
]
train[cols] = train[cols].fillna(train[cols].median())
test[cols] = test[cols].fillna(train[cols].median())

In [10]:
train['Deck'] = train['Cabin'].str[0]
test['Deck'] = train['Cabin'].str[0]


train['Deck'] = train['Deck'].fillna('U')
test['Deck'] = test['Deck'].fillna('U')

In [11]:
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])

In [12]:
train = train.drop(columns='Cabin')
test = test.drop(columns='Cabin')

In [13]:
train['Sex'] = train['Sex'].map({'male':0, 'female':1})
test['Sex'] = test['Sex'].map({'male':0, 'female':1})

In [14]:
train = train.drop(columns='Ticket')
test = test.drop(columns='Ticket')

In [15]:
train = pd.get_dummies(train, columns=['Embarked', 'Deck', 'Pclass'], drop_first=True)
test = pd.get_dummies(test, columns=['Embarked', 'Deck', 'Pclass'], drop_first=True)

In [16]:
dummy_cols = [col for col in train.columns if col.startswith('Embarked_') or col.startswith('Deck_') or col.startswith('Pclass_')]
train[dummy_cols] = train[dummy_cols].astype(int)
test[dummy_cols] = test[dummy_cols].astype(int)


In [17]:
train['Age'] = (train['Age']-train['Age'].mean())/train['Age'].std()
test['Age'] = (test['Age']-train['Age'].mean())/train['Age'].std()
train['Fare'] = (train['Fare']-train['Fare'].mean())/train['Fare'].std()
test['Fare'] = (test['Fare']-train['Fare'].mean())/train['Fare'].std()

In [18]:
x_train = train.drop(columns='Survived')
y_train = train['Survived']
x_test = test

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [20]:
def sigmoid(z):
  g = 1/(1+np.exp(-z))
  return g

In [21]:
def compute_cost(x, y, w, b):
  m, n = x.shape
  cost = 0
  for i in range(m):
    z_in = np.dot(w, x[i])+b
    f_wb = sigmoid(z_in)
    cost+=-y[i]*np.log(f_wb)-(1-y[i])*np.log(1-f_wb)
  cost = cost/m
  return cost


In [22]:
def compute_gradient_descent(x, y, w, b):
  dj_dw = np.zeros(w.shape)
  dj_db = 0
  m, n = x.shape
  for i in range(m):
    z_in = np.dot(w, x[i])+b
    f_wb = sigmoid(z_in)
    err = f_wb - y[i]
    for j in range(n):
      dj_dw[j] += err*x[i, j]
    dj_db += err
  dj_dw = dj_dw/m
  dj_db = dj_db/m
  return dj_dw, dj_db


In [23]:
def gradient_descent(X, y, w, b, alpha, num_iters):
    for i in range(num_iters):
        dj_dw, dj_db = compute_gradient_descent(X, y, w, b)

        w = w - alpha * dj_dw
        b = b - alpha * dj_db

        if i % 1000 == 0:
            cost = compute_cost(X, y, w, b)
            print(f"Iteration {i}: Cost {cost:.4f}")

    return w, b


In [24]:
X_train = X_train.to_numpy()
Y_train = Y_train.to_numpy()
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()


In [25]:
w = np.zeros(x_train.shape[1])
b = 0
w, b = gradient_descent(x_train, y_train, w, b, 0.5, 10000)

Iteration 0: Cost 0.6493
Iteration 1000: Cost 0.4304
Iteration 2000: Cost 0.4300
Iteration 3000: Cost 0.4299
Iteration 4000: Cost 0.4298
Iteration 5000: Cost 0.4297
Iteration 6000: Cost 0.4297
Iteration 7000: Cost 0.4297
Iteration 8000: Cost 0.4297
Iteration 9000: Cost 0.4297


In [26]:
print(w)
print(b)

[ 2.74602875 -0.52986082 -0.32458533 -0.10987703  0.14750132 -0.10796568
 -0.50404405  0.11315587 -0.41794212  0.56604567  1.08078254  0.38790421
 -1.45898951 -1.12679056 -0.8343089  -0.22919622 -1.39140176]
0.3953600149686645


In [27]:
y_test = sigmoid(np.dot(x_test, w) + b)
y_test = (y_test >= 0.5).astype(int)
y_test = np.array(y_test).ravel()
submission = pd.DataFrame({
    "PassengerId": passenger_ids,
    "Survived": y_test
})
submission.to_csv("submission.csv", index=False)


In [28]:
w_in = np.zeros(x_train.shape[1])
b_in = 0
w_in, b_in =  gradient_descent(X_train, Y_train, w_in, b_in, 0.5, 10000)

Iteration 0: Cost 0.6476
Iteration 1000: Cost 0.4321
Iteration 2000: Cost 0.4313
Iteration 3000: Cost 0.4310
Iteration 4000: Cost 0.4309
Iteration 5000: Cost 0.4308
Iteration 6000: Cost 0.4308
Iteration 7000: Cost 0.4308
Iteration 8000: Cost 0.4308
Iteration 9000: Cost 0.4308


In [29]:
print(w_in)
print(b_in)

[ 2.78888616 -0.41201159 -0.30580915 -0.15968265  0.21151086 -0.26322048
 -0.60908523  0.55059295 -0.05902339  0.95355517  1.61539763  0.27777895
 -2.09709261 -1.07449909 -0.93375207  0.584421   -0.64104255]
-0.08320169722541658


In [30]:
y_pred_prob = sigmoid(np.dot(w_in, X_test.T) + b_in)
y_pred = (y_pred_prob >= 0.5).astype(int)

In [31]:
accuracy = np.mean(y_pred == Y_test)
print(f'Accuracy: {accuracy*100}%')

Accuracy: 79.88826815642457%
